In [ ]:
import os
import warnings as w
w.filterwarnings('ignore')
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt

In [ ]:
train_dir = '/input/asl-alphabet/asl_alphabet_train/asl_alphabet_train'
test_dir = '/input/asl-alphabet/asl_alphabet_test'

In [ ]:
train_data, val_data = keras.utils.image_dataset_from_directory(
    train_dir, 
    labels = 'inferred',
    label_mode = 'categorical',
    validation_split = 0.25,
    subset = 'both',
    shuffle = True,
    seed = 1223,
    batch_size = 64,
    image_size = (256,256)
) 

In [ ]:
test_data = keras.utils.image_dataset_from_directory(
    test_dir, 
    labels = 'inferred',
    label_mode = 'categorical',
    image_size = (256,256)
)

In [ ]:
classes = train_data.class_names

In [ ]:
for img, label in test_data.take(1):
    for i in range(0,28):
        plt.subplot(7,4,i+1),
        plt.imshow(img[i].numpy().astype('uint'))
        plt.title(classes[np.argmax((label[i].numpy().astype('uint')))])
        plt.axis('off')

In [ ]:
test_class = test_data.class_names

In [ ]:
for img, label in train_data.take(1):
    for i in range(0,36):
        plt.subplot(6,6,i+1),
        plt.imshow(img[i].numpy().astype('uint'))
        plt.title(classes[np.argmax((label[i].numpy().astype('uint')))])
        plt.axis('off')

In [ ]:
resize_and_rescale = keras.Sequential([
    keras.layers.Resizing(256,256),
    keras.layers.Rescaling(1./255)
])

In [ ]:
conv_model = keras.Sequential([
    keras.layers.Input(shape = (256, 256, 3)),
    resize_and_rescale,
    
    keras.layers.Conv2D(32, (3,3), activation= 'relu', padding = 'same'),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPooling2D((2,2)),
    keras.layers.Dropout(0.2),

    keras.layers.Conv2D(64, (3,3), activation= 'relu', padding = 'same'),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPooling2D((2,2)),
    keras.layers.Dropout(0.3),

    keras.layers.Conv2D(128, (3,3), activation= 'relu', padding = 'same'),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPooling2D((2,2)),
    keras.layers.Dropout(0.4),

    keras.layers.Conv2D(256, (3,3), activation= 'relu', padding = 'same'),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPooling2D((2,2)),
    keras.layers.Dropout(0.5),

    keras.layers.Flatten(),
    
    keras.layers.Dense(256, activation = 'relu'),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.6),

    keras.layers.Dense(256, activation = 'relu'),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.4),
    
    keras.layers.Dense(len(classes), activation = 'softmax')
])

In [ ]:
conv_model.summary()

In [ ]:
conv_model.compile(optimizer = 'adam',
                   loss = 'categorical_crossentropy',
                   metrics = ['accuracy'])

In [ ]:
from keras.callbacks import EarlyStopping

early_stop = EarlyStopping(
    monitor = 'val_loss',
    patience = 3,
    restore_best_weights = True,
    mode = 'auto'
)

In [ ]:
history = conv_model.fit(
    train_data,
    validation_data = val_data,
    epochs = 20,
    callbacks=[early_stop],
    batch_size = 128
)

In [ ]:
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend()
plt.grid(True)
plt.show()